In [1]:
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from ehrgraphs.utils.callbacks import WritePredictionsDataFrame, WriteEmbeddingsDataFrame
from pytorch_lightning.loggers import WandbLogger
from torch_geometric import seed_everything

In [2]:
!hostname

s-sc-gpu005


In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
hydra.core.global_hydra.GlobalHydra().clear()
initialize(config_path="../../ehrgraphs/config")
args = compose(config_name="config", overrides=["model=gnn", "trainer.max_epochs=1"])
print(OmegaConf.to_yaml(args))

setup:
  entity: cardiors
  project: RecordGraphs
  name: null
  data_root:
    charite-hpc: /sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/artifacts
    eils-hpc: /data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning/artifacts
  data_identifier: WandBGraphDataNoShortcuts256:latest
  tags:
  - full_data
head:
  model_type: MLP
  dropout: 0.2
  kwargs:
    num_hidden: 256
    num_layers: 2
    detach_clf: false
datamodule:
  covariates:
  - age_at_recruitment_f21022_0_0
  - sex_f31_0_0
  sampler:
    sampler_type: DummySampler
  batch_size: 8192
  partition: 0
  num_workers: 4
  label_definition:
    all_cause_death: true
    phecodes: true
    custom: []
  t0_mode: recruitment
  use_top_n_phecodes: 1000
  edge_weight_threshold: 0.1
  min_edge_type_fraction: 0.001
  load_embeddings_path: null
  use_loss_weights: false
  task: tte
  task_kwargs: {}
model:
  model_type: GNN
  num_hidden: 256
  num_outputs: 256
  num_blocks: 3
  dr

In [4]:
from ehrgraphs.training import setup_training
seed_everything(0)

datamodule, model, tags = setup_training(args)

Using backend: pytorch


/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/artifacts/WandBGraphDataNoShortcuts256_220203.p
Generating train dataset...
Generating valid dataset...
Using edge types: ['Is a' 'ATC - RxNorm sec up' 'Tradename of' 'Has tradename' 'Constitutes'
 'RxNorm inverse is a' 'RxNorm ing of' 'Dose form group of' 'Has method'
 'Has asso morph' 'Has interprets' 'Interprets of' 'Is descendant of'
 'Is associated with' 'Is ancestor of' 'Asso morph of' 'Method of'
 'Interacts with' 'Is part of' 'Composed of']


In [5]:
wandb_logger = WandbLogger(name="debug", project=args.setup.project, tags=["debug"])
wandb_logger.watch(model, log_graph=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jakob-steinfeldt (use `wandb login --relogin` to force relogin)


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [8]:
trainer = Trainer(
    logger=wandb_logger,
    callbacks=[
        ModelCheckpoint(mode="min", monitor="valid/loss", save_top_k=1, save_last=True),
        EarlyStopping(
            monitor="valid/loss",
            min_delta=0.00000001,
            patience=50,
            verbose=False,
            mode="min",
        ),
        WritePredictionsDataFrame(),
        WriteEmbeddingsDataFrame()
    ],
    **args.trainer,
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
trainer.fit(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Using /home/steinfej/.cache/torch_extensions/py39_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/steinfej/.cache/torch_extensions/py39_cu113/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Set SLURM handle signals.

  | Name          | Type       | Params
---------------------------------------------
0 | graph_encoder | HeteroGNN  | 4.0 M 
1 | head          | MLPHead    | 456 K 
2 | valid_metrics | ModuleDict | 0     
3 | train_metrics | ModuleDict | 0     
---------------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
8.941     Total estimated model params size (MB)


ninja: no work to do.
Loading extension module fused_adam...
Time to load fused_adam op: 1.0172972679138184 seconds


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

/home/steinfej/miniconda3/envs/ehrgraphs/lib/python3.9/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Generating test dataset...


  0%|          | 0/6 [00:00<?, ?it/s]

Predictions saved /home/steinfej/code/22_medical_records/debugging/RecordGraphs/lkmzylt4/predictions/predictions.feather
Embeddings saved /home/steinfej/code/22_medical_records/debugging/RecordGraphs/lkmzylt4/embeddings/embeddings.feather


In [9]:
try:
    wandb_logger.experiment.config["best_checkpoint"] = trainer.checkpoint_callback.best_model_path
    wandb_logger.experiment.config["best_score"] = trainer.checkpoint_callback.best_model_score
except:
    pass

In [9]:
embeddings_df

NameError: name 'embeddings_df' is not defined

In [15]:
import pandas as pd
import numpy as np
x = pd.DataFrame(data=np.random.rand(16201, 256))

In [17]:
x.columns.astype(str)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '246', '247', '248', '249', '250', '251', '252', '253', '254', '255'],
      dtype='object', length=256)

In [13]:
device=torch.device("cuda:0")

In [14]:
dl = trainer.datamodule.test_dataloader()

In [15]:
ckpt = torch.load(trainer.checkpoint_callback.best_model_path)
#ckpt = torch.load("/home/steinfej/code/22_medical_records/debugging/RecordGraphs/lin2ah28/checkpoints/epoch=1-step=93.ckpt")
trainer.model.load_state_dict(ckpt['state_dict'])
trainer.model.eval()
trainer.model = trainer.model.to(device)

In [27]:
batch = next(iter(dl))

In [28]:
batch.graph = batch.graph.to(device)
batch.records = batch.records.to(device)
batch.covariates = batch.covariates.to(device)

In [29]:
preds = model(batch)

In [42]:
if "record_node_embeddings" in preds:
    embeddings = preds

In [52]:
import pandas as pd
embeddings_df = pd.DataFrame(data=embeddings, index=trainer.datamodule.record_cols).rename_axis('record_node').reset_index(drop=False)
embeddings_df

,record_node,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,OMOP_1000560,0.033274,-0.000198,-0.000073,-0.000066,0.105967,0.000138,0.000732,0.016053,0.027023,...,-0.000077,-0.000160,0.065679,0.011669,-0.000169,-0.000063,-0.000189,0.146775,0.053580,0.253157
1,OMOP_1000632,0.018683,-0.000089,-0.000076,-0.000082,0.157785,-0.000049,-0.000006,0.026211,0.039165,...,-0.000008,-0.000243,-0.000059,-0.000093,-0.000172,-0.000003,-0.000163,0.169373,-0.000157,0.199650
2,OMOP_1000772,0.034484,-0.000186,-0.000071,-0.000094,0.089039,-0.000031,0.001200,0.015641,0.031990,...,-0.000098,-0.000219,0.061523,0.029405,-0.000139,-0.000047,-0.000154,0.171055,0.043815,0.265435
3,OMOP_1000979,0.039492,-0.000170,-0.000073,-0.000060,0.091555,-0.000024,-0.000014,0.026686,0.046753,...,-0.000103,-0.000220,0.048807,0.033016,-0.000141,-0.000036,-0.000139,0.181347,0.042297,0.246745
4,OMOP_1000995,0.006587,0.000213,-0.000065,-0.000040,0.169975,-0.000032,0.006070,0.026196,0.035059,...,-0.000044,-0.000260,-0.000044,-0.000054,-0.000128,0.011278,-0.000173,0.187851,-0.000152,0.150245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16196,OMOP_997276,0.061606,-0.000138,-0.000073,-0.000082,0.097845,-0.000025,0.002858,0.030324,0.044743,...,-0.000103,-0.000225,0.046367,0.040641,-0.000140,-0.000035,-0.000173,0.186441,0.045077,0.248098
16197,OMOP_997496,0.010466,-0.000183,0.054040,-0.000085,0.133294,-0.000054,-0.000103,0.029176,0.028950,...,0.004130,-0.000178,-0.000099,-0.000089,-0.000140,-0.000111,-0.000151,0.112729,-0.000162,0.232595
16198,OMOP_997881,0.011866,-0.000097,0.031949,-0.000090,0.141077,-0.000011,-0.000030,0.027730,0.037394,...,0.003756,-0.000241,-0.000087,-0.000092,-0.000159,-0.000077,-0.000144,0.144080,-0.000157,0.220444
16199,OMOP_998394,-0.000064,-0.000192,-0.000084,-0.000072,0.143877,-0.000051,0.002316,0.013349,0.065278,...,-0.000055,-0.000204,-0.000041,-0.000036,-0.000067,-0.000051,-0.000166,0.185569,-0.000102,0.194191


In [22]:
emb1 = preds['record_node_embeddings']
emb1.shape

torch.Size([16201, 256])

In [20]:
loghs = predict_dataloader(model, dl, device)

  0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
import pandas as pd

In [22]:
eids = trainer.datamodule.eids["test"]
endpoints = list(trainer.datamodule.label_mapping.keys())

In [23]:
predictions_df = pd.DataFrame(data=loghs, index=eids, columns=endpoints)

In [24]:
predictions_df["split"] = "test"
predictions_df["partition"] = trainer.datamodule.partition
predictions_df["encoder"] = trainer.model.graph_encoder._get_name()
predictions_df["head"] = trainer.model.head._get_name()
predictions_df["covariate_cols"] = str(trainer.datamodule.covariate_cols)
predictions_df["record_cols"] = str(trainer.datamodule.record_cols)

In [25]:
for col in ["split", "partition", "encoder", "head", "covariate_cols", "record_cols"]:
    predictions_df[col] = predictions_df[col].astype("category")